# Training CNN model

In [1]:
import os

import keras
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, f1_score, precision_score, recall_score

from utils.KerasModels import buildCharCNNModel
from utils.KerasModels import generateExpData

keras.utils.set_random_seed(812)
MODELS_DIR = os.path.join('..', 'models', 'charCNN')
EPOCHS = 200

In [2]:
def prepare_data(data_directory):
    train = pd.read_csv(os.path.join(data_directory, 'train.csv'))
    validation = pd.read_csv(os.path.join(data_directory, 'validation.csv'))
    test = pd.read_csv(os.path.join(data_directory, 'test.csv'))

    dfTrainDataset = train[["screen_name", "text", "account.type"]]
    dfValDataset = validation[["screen_name", "text", "account.type"]]
    dfTestDataset = test[["screen_name", "text", "account.type"]]

    tokenizer = None
    train_features, tokenizer = generateExpData(dfTrainDataset, tokenizer=tokenizer)
    val_features, tokenizer = generateExpData(dfValDataset, tokenizer=tokenizer)
    test_features, tokenizer = generateExpData(dfTestDataset, tokenizer=tokenizer)

    dictLabels = {"human": 0, "bot": 1}
    y_train = dfTrainDataset["account.type"].apply(lambda x: dictLabels[x])
    y_val = dfValDataset["account.type"].apply(lambda x: dictLabels[x])
    y_test = dfTestDataset["account.type"].apply(lambda x: dictLabels[x])

    train_labels = y_train.tolist()
    val_labels = y_val.tolist()
    test_labels = y_test.tolist()

    vocab_size = len(tokenizer.word_index)

    return train_features, val_features, test_features, train_labels, val_labels, test_labels, vocab_size


def proba_to_pred(y_proba):
    y_pred_char_cnn = (y_proba > 0.5).astype(int)
    return y_pred_char_cnn


def calculate_metrics(y_true, y_pred):
    results = {
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred)
    }
    return results

In [3]:
DATA_PATH = os.path.join('..', 'data', 'preprocessed_url_simple')
train_features, val_features, test_features, train_labels, val_labels, test_labels, vocab_size = prepare_data(DATA_PATH)
os.makedirs(MODELS_DIR, exist_ok=True)
model = buildCharCNNModel(vocab_size, embSize=32, inputSize=320)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 320, 32)      3360        ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 318, 128)     12416       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 317, 128)     16512       ['embedding[0][0]']              
                                                                                              

In [4]:
# Train CNN model.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(os.path.join(MODELS_DIR, "simple"), monitor='val_loss',
                                            save_best_only=True, restore_best_weights=True)
model.fit(np.array(train_features), np.array(train_labels), batch_size=256, epochs=EPOCHS,
          validation_data=(np.array(val_features), np.array(val_labels)),
          callbacks=[early_stopping, save_best])

Epoch 1/200
81/81 [==============================] - ETA: 0s - loss: 0.5592 - accuracy: 0.7407

INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


81/81 [==============================] - 4s 23ms/step - loss: 0.5592 - accuracy: 0.7407 - val_loss: 0.4785 - val_accuracy: 0.7641
Epoch 2/200
79/81 [============================>.] - ETA: 0s - loss: 0.4349 - accuracy: 0.7928

INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


...

INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


81/81 [==============================] - 2s 20ms/step - loss: 0.2692 - accuracy: 0.8834 - val_loss: 0.3438 - val_accuracy: 0.8475
Epoch 21/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2661 - accuracy: 0.8830 - val_loss: 0.3443 - val_accuracy: 0.8401
Epoch 22/200
81/81 [==============================] - 1s 15ms/step - loss: 0.2573 - accuracy: 0.8896 - val_loss: 0.3529 - val_accuracy: 0.8219
Epoch 23/200
80/81 [============================>.] - ETA: 0s - loss: 0.2543 - accuracy: 0.8905

INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\simple\assets


81/81 [==============================] - 2s 20ms/step - loss: 0.2545 - accuracy: 0.8901 - val_loss: 0.3421 - val_accuracy: 0.8362
Epoch 24/200
81/81 [==============================] - 1s 14ms/step - loss: 0.2428 - accuracy: 0.8984 - val_loss: 0.3490 - val_accuracy: 0.8406
Epoch 25/200
81/81 [==============================] - 1s 15ms/step - loss: 0.2404 - accuracy: 0.8977 - val_loss: 0.3507 - val_accuracy: 0.8388
Epoch 26/200
81/81 [==============================] - 1s 14ms/step - loss: 0.2348 - accuracy: 0.9013 - val_loss: 0.3479 - val_accuracy: 0.8345
Epoch 27/200
81/81 [==============================] - 1s 15ms/step - loss: 0.2293 - accuracy: 0.9040 - val_loss: 0.3506 - val_accuracy: 0.8362
Epoch 28/200
81/81 [==============================] - 1s 14ms/step - loss: 0.2254 - accuracy: 0.9047 - val_loss: 0.3541 - val_accuracy: 0.8288
Epoch 29/200
81/81 [==============================] - 1s 15ms/step - loss: 0.2204 - accuracy: 0.9082 - val_loss: 0.3566 - val_accuracy: 0.8297
Epoch 30/200

In [5]:
probas = model.predict(np.array(test_features))
y_pred = proba_to_pred(probas)
results = calculate_metrics(test_labels, y_pred)
results

80/80 [==============================] - 0s 1ms/step


{'balanced_accuracy': 0.8396909233176839,
 'f1_score': 0.8452830188679246,
 'precision': 0.8175182481751825,
 'recall': 0.875}

# Lemmatized data 

In [6]:
LEMMATIZED_DATA_PATH = os.path.join('..', 'data', 'lemmatized')
lem_train_features, lem_val_features, lem_test_features, lem_train_labels, lem_val_labels, lem_test_labels, lem_vocab_size = prepare_data(
    LEMMATIZED_DATA_PATH)

modelLEM = buildCharCNNModel(lem_vocab_size, embSize=32, inputSize=320, verbose=False)

In [7]:
# Train CNN model.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(os.path.join(MODELS_DIR, "lemmatized"), monitor='val_loss',
                                            save_best_only=True, restore_best_weights=True)
modelLEM.fit(np.array(lem_train_features), np.array(lem_train_labels), batch_size=256, epochs=EPOCHS,
             validation_data=(np.array(lem_val_features), np.array(lem_val_labels)),
             callbacks=[early_stopping, save_best])

Epoch 1/200
80/81 [============================>.] - ETA: 0s - loss: 0.5575 - accuracy: 0.7444

INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


81/81 [==============================] - 2s 20ms/step - loss: 0.5564 - accuracy: 0.7447 - val_loss: 0.4835 - val_accuracy: 0.7615
Epoch 2/200
79/81 [============================>.] - ETA: 0s - loss: 0.4398 - accuracy: 0.7913

INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


...

INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


81/81 [==============================] - 2s 21ms/step - loss: 0.2653 - accuracy: 0.8844 - val_loss: 0.3419 - val_accuracy: 0.8414
Epoch 22/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2552 - accuracy: 0.8896 - val_loss: 0.3535 - val_accuracy: 0.8219
Epoch 23/200
79/81 [============================>.] - ETA: 0s - loss: 0.2538 - accuracy: 0.8922

INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\lemmatized\assets


81/81 [==============================] - 2s 20ms/step - loss: 0.2539 - accuracy: 0.8919 - val_loss: 0.3416 - val_accuracy: 0.8367
Epoch 24/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2425 - accuracy: 0.8989 - val_loss: 0.3453 - val_accuracy: 0.8471
Epoch 25/200
81/81 [==============================] - 1s 14ms/step - loss: 0.2400 - accuracy: 0.8990 - val_loss: 0.3449 - val_accuracy: 0.8488
Epoch 26/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2329 - accuracy: 0.9030 - val_loss: 0.3446 - val_accuracy: 0.8419
Epoch 27/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2288 - accuracy: 0.9058 - val_loss: 0.3518 - val_accuracy: 0.8445
Epoch 28/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2216 - accuracy: 0.9115 - val_loss: 0.3489 - val_accuracy: 0.8397
Epoch 29/200
81/81 [==============================] - 1s 13ms/step - loss: 0.2178 - accuracy: 0.9092 - val_loss: 0.3535 - val_accuracy: 0.8467
Epoch 30/200

In [8]:
probas_lem = modelLEM.predict(np.array(lem_test_features))
y_pred_lem = proba_to_pred(probas_lem)
results_lem = calculate_metrics(lem_test_labels, y_pred_lem)
results_lem

80/80 [==============================] - 0s 1ms/step


{'balanced_accuracy': 0.8338327709311424,
 'f1_score': 0.8384644621816799,
 'precision': 0.8164322723908216,
 'recall': 0.86171875}

# Stemmed data

In [9]:
STEMMED_DATA_PATH = os.path.join('..', 'data', 'stemmed')
stem_train_features, stem_val_features, stem_test_features, stem_train_labels, stem_val_labels, stem_test_labels, stem_vocab_size = prepare_data(
    STEMMED_DATA_PATH)

modelSTEM = buildCharCNNModel(stem_vocab_size, embSize=32, inputSize=320, verbose=False)

In [10]:
# Train CNN model.
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(os.path.join(MODELS_DIR, "stemmed"), monitor='val_loss',
                                            save_best_only=True, restore_best_weights=True)
modelSTEM.fit(np.array(stem_train_features), np.array(stem_train_labels), batch_size=256, epochs=EPOCHS,
              validation_data=(np.array(stem_val_features), np.array(stem_val_labels)),
              callbacks=[early_stopping, save_best])

Epoch 1/200
81/81 [==============================] - ETA: 0s - loss: 0.5618 - accuracy: 0.7378

INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


81/81 [==============================] - 2s 20ms/step - loss: 0.5618 - accuracy: 0.7378 - val_loss: 0.4766 - val_accuracy: 0.7667
Epoch 2/200
80/81 [============================>.] - ETA: 0s - loss: 0.4349 - accuracy: 0.7943

INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


...

INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


81/81 [==============================] - 2s 24ms/step - loss: 0.2710 - accuracy: 0.8798 - val_loss: 0.3477 - val_accuracy: 0.8445
Epoch 22/200
81/81 [==============================] - 1s 18ms/step - loss: 0.2610 - accuracy: 0.8880 - val_loss: 0.3508 - val_accuracy: 0.8271
Epoch 23/200
80/81 [============================>.] - ETA: 0s - loss: 0.2585 - accuracy: 0.8878

INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


INFO:tensorflow:Assets written to: ..\models\charCNN\stemmed\assets


81/81 [==============================] - 2s 25ms/step - loss: 0.2588 - accuracy: 0.8875 - val_loss: 0.3464 - val_accuracy: 0.8280
Epoch 24/200
81/81 [==============================] - 1s 17ms/step - loss: 0.2510 - accuracy: 0.8935 - val_loss: 0.3499 - val_accuracy: 0.8454
Epoch 25/200
81/81 [==============================] - 2s 20ms/step - loss: 0.2440 - accuracy: 0.8972 - val_loss: 0.3495 - val_accuracy: 0.8419
Epoch 26/200
81/81 [==============================] - 2s 22ms/step - loss: 0.2399 - accuracy: 0.8978 - val_loss: 0.3492 - val_accuracy: 0.8388
Epoch 27/200
81/81 [==============================] - 2s 22ms/step - loss: 0.2333 - accuracy: 0.9015 - val_loss: 0.3534 - val_accuracy: 0.8419
Epoch 28/200
81/81 [==============================] - 2s 21ms/step - loss: 0.2261 - accuracy: 0.9069 - val_loss: 0.3567 - val_accuracy: 0.8393
Epoch 29/200
81/81 [==============================] - 2s 20ms/step - loss: 0.2237 - accuracy: 0.9074 - val_loss: 0.3598 - val_accuracy: 0.8410
Epoch 30/200

In [11]:
probas_stem = modelSTEM.predict(np.array(stem_test_features))
y_pred_stem = proba_to_pred(probas_stem)
results_stem = calculate_metrics(stem_test_labels, y_pred_stem)
results_stem

80/80 [==============================] - 0s 2ms/step


{'balanced_accuracy': 0.8400986649061033,
 'f1_score': 0.8423892100192678,
 'precision': 0.8311787072243346,
 'recall': 0.85390625}

In [12]:
# Create csv with all results

results_all = pd.DataFrame([results, results_lem, results_stem])
results_all['model'] = 'CharCNN'
results_all['dataset'] = ['simple', 'lemmatized', 'stemmed']

if not os.path.exists('results'):
    os.makedirs('results')
results_all.to_csv('results/char_cnn.csv', index=False)
results_all

,balanced_accuracy,f1_score,precision,recall,model,dataset
0,0.839691,0.845283,0.817518,0.875000,CharCNN,simple
1,0.833833,0.838464,0.816432,0.861719,CharCNN,lemmatized
2,0.840099,0.842389,0.831179,0.853906,CharCNN,stemmed
